<a href="https://colab.research.google.com/github/dolmani38/QA/blob/main/Train_korQuAD2.0_0512.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#KorQuad2.0 학습.

https://github.com/huggingface/datasets

https://huggingface.co/transformers/custom_datasets.html#qa-squad



In [1]:

if True:
    from google.colab import drive
    drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install datasets

     |████████████████████████████████| 225kB 9.1MB/s 
     |████████████████████████████████| 245kB 15.2MB/s 
     |████████████████████████████████| 112kB 19.1MB/s 


In [3]:
from datasets import list_datasets, load_dataset, list_metrics, load_metric, load_from_disk

# Print all the available datasets
print(list_datasets())

# Load a dataset and print the first example in the training set
squad_dataset = load_dataset('squad_kor_v2') #,cache_dir ='/content/drive/MyDrive/korQuAD2.1/dataset')


['acronym_identification', 'ade_corpus_v2', 'adversarial_qa', 'aeslc', 'afrikaans_ner_corpus', 'ag_news', 'ai2_arc', 'air_dialogue', 'ajgt_twitter_ar', 'allegro_reviews', 'allocine', 'alt', 'amazon_polarity', 'amazon_reviews_multi', 'amazon_us_reviews', 'ambig_qa', 'amttl', 'anli', 'app_reviews', 'aqua_rat', 'aquamuse', 'ar_cov19', 'ar_res_reviews', 'ar_sarcasm', 'arabic_billion_words', 'arabic_pos_dialect', 'arabic_speech_corpus', 'arcd', 'arsentd_lev', 'art', 'arxiv_dataset', 'ascent_kb', 'aslg_pc12', 'asnq', 'asset', 'assin', 'assin2', 'atomic', 'autshumato', 'babi_qa', 'banking77', 'bbaw_egyptian', 'bbc_hindi_nli', 'bc2gm_corpus', 'best2009', 'bianet', 'bible_para', 'big_patent', 'billsum', 'bing_coronavirus_query_set', 'biomrc', 'blended_skill_talk', 'blimp', 'blog_authorship_corpus', 'bn_hate_speech', 'bookcorpus', 'bookcorpusopen', 'boolq', 'bprec', 'break_data', 'brwac', 'bsd_ja_en', 'bswac', 'c3', 'c4', 'cail2018', 'caner', 'capes', 'catalonia_independence', 'cawac', 'cbt', 'c

Dataset squad_kor_v2 downloaded and prepared to /root/.cache/huggingface/datasets/squad_kor_v2/squad_kor_v2/2.1.0/8e4ee4e5757761cf13f00b2d4e4cef2e842c0ea3c57050fec9fafc8fec60e128. Subsequent calls will reuse this data.


In [6]:
squad_dataset['train']

Dataset({
    features: ['id', 'title', 'context', 'question', 'answer', 'url', 'raw_html'],
    num_rows: 83486
})

In [7]:
squad_dataset['validation'][0]['answer']

{'answer_start': 10698, 'html_answer_start': 45900, 'text': '<td>332cm</td>'}

In [8]:
from datasets import Dataset
import pandas as pd

squad_dataset_train = {}
squad_dataset_train['id'] = []
squad_dataset_train['title'] = []
squad_dataset_train['context'] = []
squad_dataset_train['question'] = []
squad_dataset_train['answer'] = []


for sdt in squad_dataset['train']:
    squad_dataset_train['id'].append(sdt['id'])
    squad_dataset_train['title'].append(sdt['title'])
    squad_dataset_train['context'].append(sdt['context'])
    squad_dataset_train['question'].append(sdt['question'])
    squad_dataset_train['answer'].append(sdt['answer'])

dataset_train = Dataset.from_dict(squad_dataset_train)

In [9]:
len(dataset_train)

83486

In [10]:
squad_dataset_val = {}
squad_dataset_val['id'] = []
squad_dataset_val['title'] = []
squad_dataset_val['context'] = []
squad_dataset_val['question'] = []
squad_dataset_val['answer'] = []

for sdt in squad_dataset['validation']:
    squad_dataset_val['id'].append(sdt['id'])
    squad_dataset_val['title'].append(sdt['title'])
    squad_dataset_val['context'].append(sdt['context'])
    squad_dataset_val['question'].append(sdt['question'])
    squad_dataset_val['answer'].append(sdt['answer'])

dataset_val = Dataset.from_dict(squad_dataset_val)

In [11]:
len(dataset_val)

10165

In [12]:
squad_dataset = None

In [13]:
!pip install transformers

     |████████████████████████████████| 2.3MB 8.0MB/s 
     |████████████████████████████████| 3.3MB 35.7MB/s 
     |████████████████████████████████| 901kB 43.0MB/s 


In [14]:
train_contexts = dataset_train['context']
train_questions = dataset_train['question']
train_answers = dataset_train['answer']

In [17]:
len(train_contexts)

83486

In [18]:
dataset_train = None

In [19]:
val_contexts = dataset_val['context']
val_questions = dataset_val['question']
val_answers = dataset_val['answer']

In [20]:
def add_end_idx(answers, contexts):
    for answer, context in zip(answers, contexts):
        gold_text = answer['text']
        start_idx = answer['answer_start']
        end_idx = start_idx + len(gold_text)

        # sometimes squad answers are off by a character or two – fix this
        if context[start_idx:end_idx] == gold_text:
            answer['answer_end'] = end_idx
        elif context[start_idx-1:end_idx-1] == gold_text:
            answer['answer_start'] = start_idx - 1
            answer['answer_end'] = end_idx - 1     # When the gold label is off by one character
        elif context[start_idx-2:end_idx-2] == gold_text:
            answer['answer_start'] = start_idx - 2
            answer['answer_end'] = end_idx - 2     # When the gold label is off by two characters

add_end_idx(train_answers, train_contexts)
add_end_idx(val_answers, val_contexts)

In [21]:
from transformers import AutoTokenizer, DistilBertTokenizerFast, AutoModelForQuestionAnswering

SQUAD_MODEL = "monologg/koelectra-base-v3-finetuned-korquad"
STS_MODEL = "xlm-r-large-en-ko-nli-ststb"
pretraoned_kobert_model_name = "monologg/kobert"
distilled_korquad = "monologg/koelectra-small-v2-distilled-korquad-384"
tokenizer = DistilBertTokenizerFast.from_pretrained(distilled_korquad)

In [ ]:
train_encodings = tokenizer(train_contexts, train_questions, max_length=512, truncation=True, padding=True)
val_encodings = tokenizer(val_contexts, val_questions, max_length=512, truncation=True, padding=True)

In [ ]:
len(train_encodings[1])

101607

In [ ]:
def add_token_positions(encodings, answers):
    start_positions = []
    end_positions = []
    for i in range(len(answers)):
        start_positions.append(encodings.char_to_token(i, answers[i]['answer_start']))
        end_positions.append(encodings.char_to_token(i, answers[i]['answer_end'] - 1))

        # if start position is None, the answer passage has been truncated
        if start_positions[-1] is None:
            start_positions[-1] = tokenizer.model_max_length
        if end_positions[-1] is None:
            end_positions[-1] = tokenizer.model_max_length

    encodings.update({'start_positions': start_positions, 'end_positions': end_positions})


In [ ]:
add_token_positions(train_encodings, train_answers)
add_token_positions(val_encodings, val_answers)

In [ ]:
import torch

class SquadDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

    def __len__(self):
        return len(self.encodings.input_ids)

train_dataset = SquadDataset(train_encodings)
val_dataset = SquadDataset(val_encodings)

In [ ]:
len(train_dataset)

100

In [ ]:
from transformers import DistilBertForQuestionAnswering
model = DistilBertForQuestionAnswering.from_pretrained("distilbert-base-uncased")

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForQuestionAnswering: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this mode

In [ ]:
model = AutoModelForQuestionAnswering.from_pretrained("sangrimlee/bert-base-multilingual-cased-korquad")

In [ ]:
from torch.utils.data import DataLoader
from transformers import AdamW

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)
model.to(device)
model.train()

train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True)

optim = AdamW(model.parameters(), lr=5e-5)


cuda


In [ ]:
len(train_loader)

50

In [ ]:

for epoch in range(3):
    for batch in train_loader:
        optim.zero_grad()
        input_ids = batch['input_ids'].to(device)
        print('input_ids',len(input_ids))
        attention_mask = batch['attention_mask'].to(device)
        print('attention_mask',len(attention_mask))
        start_positions = batch['start_positions'].to(device)
        print('start_positions',len(start_positions))
        end_positions = batch['end_positions'].to(device)
        print('end_positions',len(end_positions))
        outputs = model(input_ids, attention_mask=attention_mask, start_positions=start_positions, end_positions=end_positions)
        loss = outputs[0]
        loss.backward()
        optim.step()

model.eval()

model.save_pretrained('/content/drive/MyDrive/korQuAD2.1/model')

RuntimeError: ignored